Ejercicio: Bosques aleatorios e hiperparámetros

El objetivo de esta unidad es explorar cómo los hiperparámetros cambian el entrenamiento y, por lo tanto, modelan el rendimiento. La línea entre la arquitectura del modelo y los hiperparámetros es un poco borrosa para los bosques aleatorios porque el entrenamiento en sí mismo cambia la arquitectura del modelo al agregar o quitar ramas.

Perseguiremos nuevamente nuestro objetivo de predecir qué delitos en San Francisco se resolverán.

Preparación de datos y entrenamiento
Carguemos nuestros datos, dividámoslos y preparémonos para el entrenamiento. Este es el mismo código que has visto en los ejercicios anteriores. Si no los ha hecho, ¡regrese y hágalos ahora!

In [ ]:
# This code is exactly the same as what we have done in the previous exercises. You do not need to read it again.

import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/san_fran_crime.csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
import graphing # custom graphing code. See our GitHub repo for details

#Import the data from the .csv file
dataset = pandas.read_csv('san_fran_crime.csv', delimiter="\t")

# One-hot encode features
dataset = pandas.get_dummies(dataset, columns=["Category", "PdDistrict"], drop_first=False)

features = [c for c in dataset.columns if c != "Resolution"]

# Make a utility method that we can re-use throughout this exercise
# To easily fit and test out model
def fit_and_test_model(model):
    '''
    Trains a model and tests it against both train and test sets
    '''  
    global features

    # Train the model
    model.fit(train[features], train.Resolution)

    # Assess its performance
    # -- Train
    predictions = model.predict(train[features])
    train_accuracy = balanced_accuracy_score(train.Resolution, predictions)

    # -- Test
    predictions = model.predict(test[features])
    test_accuracy = balanced_accuracy_score(test.Resolution, predictions)

    return train_accuracy, test_accuracy


print("Ready!")
dataset.head()

¡No olvidemos dividir nuestros datos!

In [ ]:
# Split the dataset in an 90/10 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.1, random_state=2, shuffle=True)

Criterios para dividir

El primer hiperparámetro con el que trabajaremos es el criterio. Esto es esencialmente un tipo de función de costo que se usa para determinar si un nodo debe dividirse o no. Tenemos dos opciones disponibles en el paquete que estamos usando: gini y entropía. Probemos los dos:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Shrink the training set temporarily to explore this
# setting with a more normal sample size
sample_size = 1000
full_trainset = train
train = full_trainset[:sample_size]


# Prepare the model 
rf = RandomForestClassifier(n_estimators=10,
                            # max_depth=12,
                            # max_features=cur_max_features,
                            random_state=2,
                            criterion="gini", 
                            verbose=False)
# Train and test the result
train_accuracy, test_accuracy = fit_and_test_model(rf)
# Train and test the result
print(train_accuracy, test_accuracy)

# Prepare the model 
rf = RandomForestClassifier(n_estimators=10,
                            random_state=2,
                            criterion="entropy", 
                            verbose=False)
# Train and test the result
train_accuracy, test_accuracy = fit_and_test_model(rf)
# Train and test the result
print(train_accuracy, test_accuracy)

# Roll back the train dataset to the full train set
train = full_trainset


Los resultados son sutilmente diferentes y, por lo general, solo sutilmente, ya que ambos criterios utilizan una forma similar para evaluar el desempeño. Le sugerimos que pruebe diferentes tamaños de muestra, como 50 y 50000, para ver cómo cambia esto con muestras más grandes o más pequeñas.

Disminución mínima de impurezas
La disminución mínima de impurezas es otro criterio que se utiliza para evaluar si se debe dividir un nodo. Es utilizado por los algoritmos de gini o entropía que usamos anteriormente. Si la disminución mínima de impurezas es alta, la división de un nodo debe dar como resultado una mejora sustancial del rendimiento. Si es muy bajo, los nodos se pueden dividir incluso si ofrecen muy pocas o ninguna mejora de rendimiento en el conjunto de datos de entrenamiento.

In [ ]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
full_trainset = train
train = full_trainset[:1000] # limit to 1000 samples

min_impurity_decreases = np.linspace(0, 0.0005, num=100)

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

print("Working...")
for min_impurity_decrease in min_impurity_decreases:

    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=10,
                                min_impurity_decrease=min_impurity_decrease,
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    min_impurity_decreases,
                    label_x="Minimum impurity decreases (min_impurity_decrease)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Roll back the train dataset to the full train set
train = full_trainset

Tenga en cuenta que el rendimiento del tren se reduce drásticamente a medida que somos más estrictos sobre cuándo se puede dividir un nodo. Esto se debe a que cuanto mayor sea la disminución mínima de impurezas, más estrictos seremos con respecto al cultivo de nuestro árbol. Cuanto más pequeño sea el árbol, menos sobreajuste veremos.

Los cambios en el rendimiento de la prueba son más sutiles. Un pequeño aumento por encima de cero parece aumentar el rendimiento de la prueba. Los aumentos adicionales comienzan a dañar el rendimiento de la prueba solo sutilmente.

Esto es similar a lo que vimos en el ejercicio anterior sobre el tamaño del modelo: los modelos más complejos (aquellos con más nodos) pueden ajustarse mejor a los datos de entrenamiento, pero una vez que superan cierta complejidad, comienzan a sobreajustarse.

Número máximo de características

Cuando se crean árboles, se les proporciona un subconjunto de los datos. Esto no solo significa que ven una cierta colección de filas (muestras), sino también una cierta colección de columnas (características). Cuantas más funciones se proporcionen, más probable es que un árbol dado se sobreajuste. Veamos qué sucede cuando restringimos la cantidad máxima de funciones que se pueden proporcionar a cada árbol en el bosque:

In [ ]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
full_trainset = train
train = full_trainset[:1000] # limit to 1000 samples

max_features = range(10, len(features) +1)

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

print("Working...")
for cur_max_features in max_features:
    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=50,
                                max_depth=12,
                                max_features=cur_max_features,
                                random_state=2, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    max_features,
                    label_x="Maximum number of features (max_features)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Roll back the trainset to the full set
train = full_trainset

Cuantas más funciones tengamos, peor será el sobreajuste (brecha entre las líneas azul y roja). Los aumentos iniciales de 10 a 20 brindan una mejora mínima en el desempeño de la prueba, después de lo cual comienza a afectar muy levemente el desempeño de la prueba. A medida que aumentan las funciones, el rendimiento del entrenamiento continúa creciendo, sin igual por el rendimiento de la prueba, lo que indica un sobreajuste. Un valor óptimo aquí sería alrededor de 20 funciones.

siembra
Finalmente, llegamos a la siembra. Cuando los árboles se crean inicialmente, hay un grado de aleatoriedad que se usa para decidir qué características y muestras se proporcionarán a qué árboles. Cambiar el valor del estado aleatorio (semilla) cambia este estado inicial.

La semilla aleatoria no es un parámetro que deba ajustarse, pero sus efectos en nuestros modelos no deben olvidarse, especialmente cuando no hay muchos datos con los que trabajar. Veamos cómo se comporta nuestro modelo con diferentes estados aleatorios.

In [ ]:
# Shrink the training set temporarily to explore this
# setting with a more normal sample size
sample_size = 1000
full_trainset = train
train = full_trainset[:sample_size] 


seeds = range(0,101)

# Train our models and report their performance
train_accuracies = []
test_accuracies = []

for seed in seeds:
    # Prepare the model 
    rf = RandomForestClassifier(n_estimators=10,
                                random_state=seed, 
                                verbose=False)
    
    # Train and test the result
    train_accuracy, test_accuracy = fit_and_test_model(rf)

    # Save the results
    test_accuracies.append(test_accuracy)
    train_accuracies.append(train_accuracy)


# Plot results
graphing.line_2D(dict(Train=train_accuracies, Test=test_accuracies), 
                    seeds,
                    label_x="Seed value (random_state)",
                    label_y="Accuracy",
                    title="Performance", show=True)

# Roll back the trainset to the full set
train = full_trainset

El rendimiento, particularmente en el conjunto de prueba, es variable y, por lo tanto, una parte del rendimiento es pura suerte. Esto no solo se debe a que el estado inicial del modelo puede ser diferente, sino también a que dividimos nuestros datos de entrenamiento y prueba de manera diferente. No es fácil saber si esto se aplicaría a un conjunto reservado sin probarlo.

No existe una correlación entre los valores de semilla altos o bajos y el rendimiento: la semilla no es algo para 'afinar'. La semilla es un factor aleatorio y puede ayudar o dificultar dependiendo del modelo en juego. En términos generales, cuando trabajamos con pequeñas cantidades de datos, es más probable que nos veamos afectados por diferentes valores semilla. Los modelos más complejos también pueden verse más afectados por la semilla, pero no siempre.

Intente cambiar el tamaño de la muestra y/o la cantidad de árboles en el modelo anterior y observe cómo cambia la variabilidad en el rendimiento. Piensa por qué podría ser esto.

Resumen
Los modelos complejos suelen tener hiperparámetros asociados que pueden afectar el entrenamiento. La medida en que estos importan y cómo afectan el resultado depende de los datos disponibles y la complejidad del modelo que se utiliza. Por lo general, necesitamos experimentar un poco con estos para lograr un rendimiento óptimo para los datos que tenemos.